# Midinet Music Generator
## Setting up
First we install the required dependencies through pip.

In [7]:
pip install mido

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.1/51.1 kB 3.3 MB/s eta 0:00:00


Then mount to Google Drive and change the directory to runescape-midinet or where ever the project root folder is located.

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
cd drive/MyDrive/midinet-lo-fi

/content/drive/MyDrive/midinet-lo-fi


Now we can import the methods we need.

In [8]:
from train import train
from generate import generate
from preprocess import process_midis
from download import download_dataset
import json

## Downloading the datasets

To start off, we need to download our datasets. We will be using two different datasets: the Runescape OST and the Lahk MIDI Dataset.

Arguments:
* dataset: the dataset to download (rs_ost or lahk)
* output_directory: the directory for the downloaded dataset

In [ ]:
dataset = 'rs_ost'
output_directory = 'rs_ost_raw'

download_dataset(dataset, output_directory)

In [ ]:
dataset = 'lahk'
output_directory = 'lahk_raw'

download_dataset(dataset, output_directory)

## Preprocessing the datasets

Before we start start training, we must preprocess the MIDIs from the downloaded datasets into a tokenized sequence.

*(note: this may take several hours depending on the size of the dataset)*

Arguments:
* input_directory: the directory containing the downloaded MIDI files
* output_directory: the directory to save the preprocessed dataset to

In [ ]:
input_directory = 'rs_ost_raw'
output_directory = 'rs_ost'

process_midis(input_directory, output_directory)

converting: 100%|██████████| 93/93 [00:22<00:00,  4.09it/s]


Skipped 0 files.
Saved to 'rs_ost'.


In [ ]:
input_directory = 'lahk_raw'
output_directory = 'lahk'

process_midis(input_directory, output_directory)

converting: 100%|██████████| 242/242 [02:23<00:00,  1.69it/s]


Skipped 7 files.
Saved to 'lahk'.


## Model Parameters

These are the parameters that control the model:

* vocab_size: the size of the vocabulary (do not change)
* embedding_size: the size of embedding layer
* sequence_length: the length of the input sequence
* num_blocks: the number of decoder blocks
* num_heads: the number of attention heads
* feed_forward_dim: the dimension size for the feed forward layers
* dropout: the dropout rate
* learning_rate: the learning rate
* batch_size: the batch size used during training

In [11]:
parameters = {
    "vocab_size": 517,
    "embedding_size": 512,
    "sequence_length": 2048,
    "num_blocks": 6,
    "num_heads": 8,
    "feed_forward_dim": 1024,
    "dropout": 0.010,
    "learning_rate": 0.0001,
    "batch_size": 2
}

with open('parameters.json', 'w') as f:
    json.dump(parameters, f)

## Training

Once we have our datasets ready, we can start training our model.

TensorBoard graphs to track loss and accuracy can be under the `logs` directory.
After training is finished, the model will be saved to the directory specified by `save_directory`.

Arguments:
* epochs: number of epochs to train the model for
* save_directory: the directory the model will be saved to
* dataset: the directory containing the dataset to train on
* restore_from_checkpoint: whether to restore from the last checkpoint

In [15]:
epochs = 1
save_directory = 'midinet_model_lo'
dataset = 'rs_ost'
restore_from_checkpoint = True

train(epochs, save_directory, dataset, restore_from_checkpoint)

Using dataset 'rs_ost.'
-Model Parameters-
vocab_size: 517
embedding_size: 512
sequence_length: 2048
num_blocks: 6
num_heads: 8
feed_forward_dim: 1024
dropout: 0.01
learning_rate: 0.0001
batch_size: 2
Restoring from latest checkpoint.
42/42 [==============================] - ETA: 0s - loss: 0.1281 - accuracy: 0.9647New best val_loss!
Current best val_loss: 0.2968
42/42 [==============================] - 55s 707ms/step - loss: 0.1281 - accuracy: 0.9647 - val_loss: 0.2968 - val_accuracy: 0.9332
TensorBoard logs can be found under 'logs/24-May-2023_09.10.11'.
Saving model as 'midinet_model_lo'.


## Inference

After training, we can use the model for inference to generate MIDI music. The model will be given random samples from the dataset as the seed to begin inference with.

To generate some music, simply run the cell below. The model that will be used for inference is specified by `save_directory`. After it finishes, the generated MIDI will be saved under the `output` directory.

*(note: make sure the sequence length specified in model parameters matches the sequence length for the model being used)*

Arguments:
* song_length: the number of song tokens to be generated
* top_k: the k value for selecting top k predictions
* save_directory: the directory to load the saved model from
* dataset: the directory containing the dataset to draw a starting seed from
* inclusive: whether or not to include the starting seed in the generated MIDI

In [ ]:
song_length = 2500
top_k = 8
save_directory = 'midinet_model'
dataset = 'lahk'
inclusive = False

generate(song_length, top_k, save_directory, dataset, inclusive)


Interferenza su altro dataset




In [ ]:
song_length = 250000
top_k = 8
save_directory = 'midinet_model_lo'
dataset = 'lahk'
inclusive = False

generate(song_length, top_k, save_directory, dataset, inclusive)

Using dataset 'lahk.'


generating:   0%|          | 0/250000 [00:00<?, ?it/s]

1/1 [==============================] - 2s 2s/step


generating:   0%|          | 1/250000 [00:01<134:44:28,  1.94s/it]

1/1 [==============================] - 0s 212ms/step


...

1/1 [==============================] - 0s 190ms/step


generating:   2%|▏         | 5876/250000 [31:10<22:06:14,  3.07it/s]

1/1 [==============================] - 0s 177ms/step


generating:   2%|▏         | 5877/250000 [31:11<23:24:36,  2.90it/s]

1/1 [==============================] - 0s 160ms/step
